# Assignment 7

### Part A

In [1]:
import numpy as np
import pandas as pd

Consider the following way of representing a portfolio. Each row in the DataFrame represents one security, with information given about the security's beta, volatility, and amount invested in that security. Note that there is not guaranteed to be 2 securities in the portfolio.

In [2]:
portfolio = pd.DataFrame(data = {"Beta": [1.2, 1.4], 
                                 "Volatility": [0.25, 0.4],
                                 "Amount Invested": [4000, 6000] })



portfolio

Beta  Volatility  Amount Invested
0   1.2        0.25             4000
1   1.4        0.40             6000

Write the function ```expected_return```, which calculates the expected return of the portfolio ```P```. There is a risk free asset with risk free rate ```rf```, and the market risk premium is ```mkt_rp```. 

expected return: expected_return(portfolio, 0.02, 0.05) should yield 0.086. 

In [3]:
def expected_return(P, rf, mkt_rp):
    # Write your code here
    total_amount = P['Amount Invested'].sum()
    
    sum = 0
    for i in range(len(P)):
        sum += P.loc[i,'Beta'] * P.loc[i, 'Amount Invested'] / total_amount
        
    exp_ret = rf + mkt_rp * sum
    
    return exp_ret

Write the function ```volatility```, which calculates the volatility of the portfolio. You can assume that the correlation between all (different) pairs of stocks in the portfolio is ```cor```.

expected return: volatility(portfolio, -0.2) should yield 0.2408318915758459

In [5]:
def volatility(P, cor):
    # Write your code here

    total_amount = P['Amount Invested'].sum()
    
    sum = 0
    for i in range(len(P)):
        sum += ((P.loc[i,'Amount Invested']/ total_amount)**2) * (P.loc[i, 'Volatility']**2)
    
    product_percentage = 1
    for i in range(len(P)):
        product_percentage *= (P.loc[i,'Amount Invested']/ total_amount)
    product_volatility = 1
    for i in range(len(P)):
        product_volatility *= P.loc[i, 'Volatility']
    num_stocks = len(P)
    
    p_vol = np.sqrt(sum + num_stocks * product_percentage * product_volatility * cor)
    
    return p_vol

Suppose there is a risk-free asset with return of ```rf```, the market risk premium is ```mkt_rp```, and the volatility of returns on the market is ```mkt_vol```.

Write the function ```expected_return2```, which creates a portfolio with volatility ```vol``` using (only) the risk free asset and the market portfolio, and calculates the expected return of that portfolio.

expected return: expected_return2(0.02, 0.05, 0.16, 0.2408) should yield 0.09525

In [7]:
def expected_return2(rf, mkt_rp, mkt_vol, vol):
    # Write your code here
    
    beta = vol / mkt_vol
  
    exp_ret_2 = rf + beta * mkt_rp
    
    return exp_ret_2


### Part B

Write the function ```CAPM_beta```, which will consume a **filepath** ```return_path```, (formatted as the provided "monthlycsrp_mod7.csv"), and a **filepath** `factors_path` (formatted as the provided "F-F_Research_Data_Factors.CSV"). This function will return a DataFrame formatted like the one supplied. Each row in the returned DataFrame should correspond with 1 CUSIP, its expected return, and its CAPM-beta.

Please note that monthlycsrp_mod7.csv's return data are in decimals. "F-F_Research_Data_Factors.CSV" has data in percentages. More details of "F-F_Research_Data_Factors.CSV" are available on https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_factors.html. 

Also, note that firm i's CAPM-beta is calculated as Cov(R_i-Rf, R_mkt – Rf)/Var(R_mkt-Rf). 

expected return: it should return output of 3225 rows × 3 columns

In [9]:
def CAPM_beta(return_path, factors_path):
    #Write your code here
    
    # read data files
    return_data = pd.read_csv(return_path)
    factors_data = pd.read_csv(factors_path)
    
    # turn return tate into percentage
    return_data['RET'] = return_data['RET'] * 100
    
    # concat return_data's the month and year column into one column
    return_data['month'] = return_data.month.map("{:02}".format)
    return_data['year'] = return_data['year'].astype("string")
    return_data['month'] = return_data['month'].astype("string")
    return_data['period'] = return_data.year.str.cat(return_data.month)
    
    # change factor_data's date into the same format as return_data
    factors_data['date'] = factors_data['date'].astype("string")
    
    # merge the two dataframe by date
    merged_data = pd.merge(return_data, factors_data, left_on=['period'], right_on=['date'])
    merged_data['ind_r_p'] = merged_data['RET']- merged_data['RF']
        
    result_df = merged_data.groupby('CUSIP').apply(lambda group: pd.Series({
        'BETA': (group['ind_r_p'].cov(group['Mkt-RF'])) / group['Mkt-RF'].var()
    })).reset_index()
        
    output = pd.DataFrame(data = {
        "EXP_RETURN" : merged_data.groupby('CUSIP')['RET'].mean()
    }).reset_index()
    
    final = pd.merge(result_df, output, left_on=['CUSIP'], right_on=['CUSIP'])    
    
    return final


Write the function ```slope```, which will return the slope of the security market line. This function will take in a **DataFrame** ```data```, whose format is that of the output of ```CAPM_beta```. You may find the function `np.polyfit` useful.

In [11]:
def slope(data):
    # This should return 0.007981588120535642... with the provided data.
    coefficients = np.polyfit(data['BETA'], data['EXP_RETURN'], 1)
    
    # The slope (market risk premium) is the first coefficient
    slope = coefficients[1] * 0.01
    
    return slope